# Pneumonia Detection using MobileNetV2

In [ ]:

# Pneumonia Detection with MobileNetV2

## 📦 Library Imports


# ======================
# PARAMETERS
# ======================
BATCH_SIZE = 48
image_size = 224
image_height = 224
EPOCHS = 2

# ======================
# DATA AUGMENTATION & GENERATORS
# ======================
data_generator_1 = ImageDataGenerator(
    rescale=1./255,
    rotation_range=5,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.05,
    zoom_range=0.05,
    brightness_range=[0.95, 1.05],
    fill_mode='nearest'
)

data_generator_3 = ImageDataGenerator(rescale=1./255)

# raw_generator: class bilgilerini almak için
raw_generator = data_generator_1.flow_from_directory(
    directory='./dataset/chest_xray/train',
    color_mode='rgb',
    target_size=(image_height, image_size),
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42
)

dict_class = raw_generator.class_indices
class_names = list(dict_class.keys())
y_classes = raw_generator.classes

# class weight hesapla
class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(y_classes),
                                     y=y_classes)
class_weights = dict(zip(np.unique(y_classes), class_weights))
print("Class weights:", class_weights)

# Pie chart
freq = np.unique(y_classes, return_counts=True)
plt.title('Training Dataset Class Distribution', fontsize=16)
plt.pie(freq[1], labels=class_names, autopct='%1.0f%%')
plt.show()

print('Train Dataset:')
print('Number of images:', len(y_classes))
print('Number of normal images:', np.sum(y_classes == 0))
print('Number of pneumonia images:', np.sum(y_classes == 1))

# test set
test_generator = data_generator_3.flow_from_directory(
    directory='./dataset/chest_xray/test',
    color_mode='rgb',
    target_size=(image_height, image_size),
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False,
    seed=42
)

print('Test Dataset:')
print('Number of images:', len(test_generator.classes))
print('Number of normal images:', np.sum(test_generator.classes == 0))
print('Number of pneumonia images:', np.sum(test_generator.classes == 1))

# Görsel örnek gösterimi
images, labels = raw_generator.next()
plt.figure(figsize=(10, 5))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(images[i])
    plt.axis('off')
    plt.title(class_names[np.argmax(labels[i])])
plt.show()

# ======================
# convert to tf.data.Dataset for performance
# ======================
train_generator_1 = tf.data.Dataset.from_generator(
    lambda: raw_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, image_height, image_size, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 2), dtype=tf.float32)
    )
).prefetch(tf.data.AUTOTUNE)

# ======================
# Model & Training
# ======================
strategy = tf.distribute.MirroredStrategy(devices=['/gpu:0', '/gpu:1'])

with strategy.scope():
    base_model = MobileNetV2(weights='imagenet',
                             include_top=False,
                             input_shape=(image_height, image_size, 3))
    base_model.trainable = False

    model_MobileNetV2 = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        Dropout(0.4),
        Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        Dropout(0.4),
        Dense(2, activation='softmax')
    ])

    model_MobileNetV2.compile(optimizer='adam',
                              loss='categorical_crossentropy',
                              metrics=['accuracy'])

    early_stopping = EarlyStopping(monitor='val_accuracy', patience=2, verbose=1, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.001, patience=10, verbose=1)

    model_MobileNetV2.summary()

    history_MobileNet = model_MobileNetV2.fit(
        train_generator_1,
        epochs=EPOCHS,
        validation_data=test_generator,
        callbacks=[early_stopping, reduce_lr],
        class_weight=class_weights,
        workers=4,
        use_multiprocessing=True
    )

    val_loss_MobileNet, val_accuracy_MobileNet = model_MobileNetV2.evaluate(test_generator, steps=len(test_generator))
    print(f"Validation Loss: {val_loss_MobileNet:.4f}")
    print(f"Validation Accuracy: {val_accuracy_MobileNet:.4f}")

# ======================
# Evaluation & Visualization
# ======================
data = {'MobileNetV2': val_accuracy_MobileNet}
df = pd.DataFrame.from_dict(data, orient='index', columns=['Accuracy']).reset_index().rename(columns={'index': 'Model'})
sns.barplot(x='Model', y='Accuracy', data=df)
plt.title('Model Accuracy')
plt.tight_layout()
plt.show()

# Fine-Tuning
no_base_layers = len(model_MobileNetV2.layers)
no_finetune_layers = int(no_base_layers / 2)

print(f'No. of base layers in the model = {no_base_layers}')
print(f'No. of layers for fine-tune = {no_finetune_layers}')

model_MobileNetV2.trainable = True
for layer in model_MobileNetV2.layers[:-no_finetune_layers]:
    layer.trainable = False

model_MobileNetV2.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
                          loss='categorical_crossentropy',
                          metrics=['accuracy'])

history_finetune_MobileNetV2 = model_MobileNetV2.fit(
    train_generator_1,
    epochs=EPOCHS,
    validation_data=test_generator,
    callbacks=[early_stopping]
)

val_loss_finetune, val_accuracy_finetune = model_MobileNetV2.evaluate(test_generator, steps=len(test_generator))
print(f"Validation Loss after Fine-tuning: {val_loss_finetune:.4f}")
print(f"Validation Accuracy after Fine-tuning: {val_accuracy_finetune:.4f}")

# ======================
# Prediction Visualization
# ======================
test_generator.reset()
img_batch, label_batch = next(iter(test_generator))

predictions = model_MobileNetV2.predict(img_batch)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(label_batch, axis=1)

plt.figure(figsize=[14, 14])
for i in range(20):
    plt.subplot(5, 4, i+1)
    plt.imshow(img_batch[i])
    plt.axis('off')
    true_label = class_names[true_classes[i]]
    predicted_label = class_names[predicted_classes[i]]
    confidence = 100 * np.max(predictions[i])
    plt.title(f"Label: {true_label}\nPrediction: {predicted_label} ({confidence:.1f}%)")
plt.tight_layout()
plt.show()
